In [26]:
import asyncio
import json
import os
from datetime import datetime, UTC

import websockets
from dotenv import load_dotenv

# --- Configuration ---
WS_URL = "ws://localhost:2025"   # 🔁 replace with actual URL
CONNECT_TIMEOUT = 10             # seconds to wait when opening
RECV_TIMEOUT = 2                 # timeout per recv loop iteration
KEEP_ALIVE = True                # keep connection open between messages

load_dotenv()
if not os.environ.get("OPENAI_API_KEY"):
    print(f"[{datetime.now(UTC).isoformat()}] ⚠️ OPENAI_API_KEY was not found in the environment.")


def ts() -> str:
    """Return a short UTC timestamp for log lines."""
    return datetime.now(UTC).strftime("%H:%M:%S")


print(f"[{datetime.now(UTC).isoformat()}] ✅ Configured Wordle listener → {WS_URL}")

[2025-11-14T11:56:53.720323+00:00] ✅ Configured Wordle listener → ws://localhost:2025


In [27]:
from dataclasses import dataclass
from typing import List, Optional

def map_wordle_feedback_token(token: str) -> str:
    cleaned = str(token).strip().lower()
    if cleaned in {"correct", "green", "g"}:
        return "correct"
    if cleaned in {"present", "yellow", "y"}:
        return "present"
    return "absent"

@dataclass
class ParsedMessage:
    raw: str
    type: Optional[str]
    command: Optional[str]
    match_id: Optional[str]
    game_id: Optional[str]
    your_id: Optional[str]
    otp: Optional[str]
    word_length: Optional[int]
    max_attempts: Optional[int]
    last_guess: str
    last_result: List[str]
    current_attempt: Optional[int]
    ack_for: Optional[str]
    ack_data: Optional[str]
    result: Optional[str]
    word: Optional[str]

def parse_message(text: str) -> Optional[ParsedMessage]:
    """Return a ParsedMessage with only the fields the bot actually needs."""
    try:
        obj = json.loads(text)
    except json.JSONDecodeError:
        return None

    last_guess = obj.get("lastGuess") or ""
    raw_result = obj.get("lastResult") or []
    if isinstance(raw_result, list):
        normalized = [map_wordle_feedback_token(s) for s in raw_result]
    else:
        normalized = []

    return ParsedMessage(
        raw=text,
        type=obj.get("type"),
        command=obj.get("command"),
        match_id=obj.get("matchId"),
        game_id=obj.get("gameId"),
        your_id=obj.get("yourId"),
        otp=obj.get("otp"),
        word_length=obj.get("wordLength"),
        max_attempts=obj.get("maxAttempts"),
        last_guess=last_guess,
        last_result=normalized,
        current_attempt=obj.get("currentAttempt"),
        ack_for=obj.get("ackFor"),
        ack_data=obj.get("ackData"),
        result=obj.get("result"),
        word=obj.get("word"),
    )

### OpenAI Models
- OpenAI models evolve quickly, and understanding their capabilities helps you balance accuracy, latency, and cost.
- Reviewing model pricing and use-case fit up front keeps you from exhausting your budget on the wrong tier.
- Learn more about token limits, model selection tips, and deployment guides in [OpenAI Bytes](https://spl.solitontech.ai/docs/learning/openai-bytes).

In [28]:
from typing import Optional

from openai import OpenAI
from pydantic import BaseModel

AI_MODEL = "gpt-5-nano"          # specify the AI model to use, switch it up and have fun
USE_STRUCTURED_OUTPUT = True  # toggle between structured vs plain LLM guesses


class GuessWord(BaseModel):
    guess: str


_ai_client: Optional[OpenAI] = None


def _get_client() -> Optional[OpenAI]:
    """Lazy-load the OpenAI client so the notebook only instantiates it when needed."""
    global _ai_client
    if _ai_client is None:
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            print(f"[{ts()}] ⚠️ OPENAI_API_KEY is not set. Skipping AI client initialization.")
            return None
        _ai_client = OpenAI(api_key=api_key)
    return _ai_client


In [29]:


def _extract_guess_text(response) -> str:
    text_parts: List[str] = []
    for item in getattr(response, "output", []):
        if getattr(item, "type", "") != "message":
            continue
        for content in getattr(item, "content", []):
            if getattr(content, "type", "") == "output_text":
                text_parts.append(getattr(content, "text", ""))
    return " ".join(text_parts).strip()


def _log_token_usage(response) -> None:
    usage = getattr(response, "usage", None)
    if not usage:
        return

    if isinstance(usage, dict):
        input_tokens = usage.get("input_tokens")
        output_tokens = usage.get("output_tokens")
    else:
        input_tokens = getattr(usage, "input_tokens", None)
        output_tokens = getattr(usage, "output_tokens", None)

    if input_tokens is None and output_tokens is None:
        return

    print(f"[{ts()}] 📊 Token usage — input: {input_tokens} • output: {output_tokens}")



In [30]:


def _ai_guess_simple(length: int) -> Optional[str]:
    """Ask the model for the next guess using plain text responses."""
    client = _get_client()
    if client is None:
        return None

    # system prompt sets the long-lived assistant role (tone & mission)
    system_prompt = "You are playing a game."
    # user prompt delivers the turn-specific instruction
    user_prompt = f"Return only one lowercase {length}-letter guess."
    try:
        response = client.responses.create(
            model=AI_MODEL,
            input=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": user_prompt,
                },
            ],
            reasoning={ "effort": "low" },
            text={ "verbosity": "low" },
        )
    except Exception as exc:
        print(f"[{ts()}] ⚠️ AI guess failed: {exc}")
        return None

    _log_token_usage(response)

    raw_text = _extract_guess_text(response)
    guess = raw_text.split()[0].lower() if raw_text else ""

    if not guess:
        print(f"[{ts()}] ⚠️ AI response was empty; falling back to deterministic guess.")
        return None
    return guess



### Structured Responses for Guesses

- OpenAI models can return structured JSON so our bot stays on script every turn.
- By pairing the Responses `parse` helper with a lightweight Pydantic model, we guarantee each reply includes a next guess plus a concept refresher—perfect for mission debriefs and quick debugging.

**Why Structured Output Matters**
- Keeps responses machine-readable, preventing brittle string parsing in the bot.
- Reduces hallucinated formats, so automations stay resilient even with generative models.
- Enables guardrails and validation paths to catch malformed payloads before they impact gameplay.
- Makes telemetry and iteration straightforward because every turn produces comparable data.

Learn more about [Structured Outputs in OpenAI](https://platform.openai.com/docs/guides/structured-outputs)

In [31]:
def _ai_guess_structured(length: int) -> Optional[str]:
    """Ask the model for the next guess using structured output parsing."""
    client = _get_client()
    if client is None:
        return None

    prompt = f"Return only one lowercase {length}-letter guess."
    system_prompt = "You are playing a game."
    try:
        response = client.responses.parse(
            model=AI_MODEL,
            input=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            reasoning={ "effort": "low" },
            text={ "verbosity": "low" },
            text_format=GuessWord,
        )
    except Exception as exc:
        print(f"[{ts()}] ⚠️ AI guess failed: {exc}")
        return None

    _log_token_usage(response)

    parsed_payload = getattr(response, "output_parsed", None)
    if not parsed_payload:
        print(f"[{ts()}] ⚠️ AI response returned no payload; switching to fallback.")
        return None

    guess = parsed_payload.guess.strip().lower()

    if not guess:
        print(f"[{ts()}] ⚠️ AI response was empty; falling back to deterministic guess.")
        return None
    return guess




In [32]:

def _ai_guess(length: int) -> Optional[str]:
    if USE_STRUCTURED_OUTPUT:
        return _ai_guess_structured(length)
    return _ai_guess_simple(length)



In [33]:

def make_guess(parsed: ParsedMessage) -> Optional[str]:
    """
    Use OpenAI to choose the next guess while learning from prior feedback.
    """
    if parsed.command != "guess":
        return None

    length = parsed.word_length or 5
    guess = _ai_guess(length)
    if guess:
        return guess

    alphabet = "abcdefghijklmnopqrstuvwxyz"
    fallback = alphabet[:length]
    print(f"[{ts()}] ✳️ Using alphabet fallback guess: {fallback}")
    return fallback

In [34]:
from typing import Any

def build_response(parsed: ParsedMessage, guess: Optional[str]) -> Optional[dict[str, Any]]:
    """
    Frames the response in the expected shape for a guess command.
    Expected fields (from prior examples): matchId, gameId, otp, guess
    If any critical field is missing, return None (we won't send).
    """
    if parsed.command != "guess" or not guess:
        return None

    if not parsed.match_id or not parsed.game_id or not parsed.otp:
        # We need these to respond correctly
        return None

    return {
        "matchId": parsed.match_id,
        "gameId": parsed.game_id,
        "otp": parsed.otp,
        "guess": guess,
    }

In [35]:
async def connect_parse_respond_forever():
    print(f"[{ts()}] 🔌 Connecting to {WS_URL} ...")
    try:
        async with websockets.connect(WS_URL, open_timeout=CONNECT_TIMEOUT) as ws:
            print(f"[{ts()}] ✅ Connection established.")
            print(
                f"[{ts()}] 👂 Listening for guess commands only (timeout={RECV_TIMEOUT}s)."
            )

            while True:
                try:
                    msg = await asyncio.wait_for(ws.recv(), timeout=RECV_TIMEOUT)

                    parsed = parse_message(msg)
                    if parsed is None:
                        print(f"[{ts()}] ⚠️ Incoming text was not JSON; ignoring.")
                        continue

                    if parsed.type == "game result":
                        outcome = parsed.result or "no outcome provided"
                        correct_word = parsed.word
                        print(f"[{ts()}] 🎯 Game result : {outcome}. Correct Word: {correct_word}")
                        continue

                    # print(f"[{ts()}] 📝 Parsed message: {parsed}")
                    guess = make_guess(parsed)
                    if guess:
                        print(f"[{ts()}] 🧠 Proposed guess: {guess}")

                    resp = build_response(parsed, guess)
                    if resp:
                        await ws.send(json.dumps(resp))
                        # print(f"[{ts()}] 📤 Sent guess response: {resp}")

                except asyncio.TimeoutError:
                    if KEEP_ALIVE:
                        continue
                    print(
                        f"[{ts()}] ⏹️ No messages within {RECV_TIMEOUT}s; closing connection."
                    )
                    break
                except websockets.exceptions.ConnectionClosedOK:
                    print(f"[{ts()}] 🔒 Connection closed by server (OK).")
                    break
                except websockets.exceptions.ConnectionClosedError as e:
                    print(f"[{ts()}] ❌ Connection closed with error: {e}")
                    break
                except Exception as e:
                    print(f"[{ts()}] ⚠️ Unexpected error while listening: {e}")
                    break

    except Exception as e:
        print(f"[{ts()}] ❌ Connection failed: {e}")

In [39]:
# Run it (infinite loop until you interrupt the cell)
await connect_parse_respond_forever()

[12:19:11] 🔌 Connecting to ws://localhost:2025 ...
[12:19:13] ✅ Connection established.
[12:19:13] 👂 Listening for guess commands only (timeout=2s).
[12:23:21] 📊 Token usage — input: 56 • output: 144
[12:23:21] 🧠 Proposed guess: crane
[12:23:53] 📊 Token usage — input: 56 • output: 79
[12:23:53] 🧠 Proposed guess: plant
[12:23:56] 📊 Token usage — input: 56 • output: 144
[12:23:56] 🧠 Proposed guess: crane
[12:23:58] 📊 Token usage — input: 56 • output: 208
[12:23:58] 🧠 Proposed guess: crane
[12:24:01] 📊 Token usage — input: 56 • output: 144
[12:24:01] 🧠 Proposed guess: crane
[12:24:34] 📊 Token usage — input: 56 • output: 80
[12:24:34] 🧠 Proposed guess: crane
[12:24:37] 📊 Token usage — input: 56 • output: 144
[12:24:37] 🧠 Proposed guess: crane
[12:24:39] 📊 Token usage — input: 56 • output: 144
[12:24:39] 🧠 Proposed guess: crane
[12:24:41] 📊 Token usage — input: 56 • output: 144
[12:24:41] 🧠 Proposed guess: crane
[12:24:44] 📊 Token usage — input: 56 • output: 144
[12:24:44] 🧠 Proposed guess:

CancelledError: 